> [Kaggle入门教程](http://www.cnblogs.com/cutd/p/5713520.html)

# 查看数据

## 属性解释
- PassengerId：一个用以标记每个乘客的数字id
- Survived：标记乘客是否幸存——幸存(1)、死亡(0)。我们将预测这一列。
- Pclass：标记乘客所属船层——第一层(1),第二层(2),第三层(3)。
- Name：乘客名字。
- Sex：乘客性别——男male、女female
- Age：乘客年龄。部分。
- SibSp：船上兄弟姐妹和配偶的数量。
- Parch：船上父母和孩子的数量。
- Ticket：乘客的船票号码。
- Fare：乘客为船票付了多少钱。
- Cabin：乘客住在哪个船舱。
- Embarked：乘客从哪个地方登上泰坦尼克号。

我们都知道妇女和儿童更可能被救。因此，年龄和性别很可能更好的帮助我们预测。认为乘客的船层可能会影响结果也是符合逻辑的，因为第一层的船舱更靠近船的甲板。Fare票价和乘客所住船层相关，而且可能是高度相关的，但是也可能会增加一些额外的信息。SibSp、Parch兄弟姐妹、配偶、父母/孩子的数量很可能关系到是否被某一个或很多个人救，会有很多人去帮助你或者有很多人想到你尝试去救你。
像Embarked登船(也许有一些信息和怎么靠近船的顶部的人的船舱有关),Ticket票号和Name名字。
这一步通常是习得相关的领域知识[要对业务比较深入]，这对于绝大多数机器学习任务来说非常非常非常的重要。

In [1]:
import pandas as pd
# We can use the pandas library in python to read in the csv file.
# This creates a pandas dataframe and assigns it to the titanic variable.
titanic = pd.read_csv("../input/train.csv")

print(titanic.describe())  # 只显示数值列

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


In [2]:
print(titanic.head(3))

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  


# 数据清洗
- 使用中位数填充年龄缺失值

In [3]:
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].median())

## 非数值列
- Name, Sex, Cabin, Embarked, Ticket
- 忽略Ticket, Cabin,Name，因为Cabin缺失太严重，而且起初看起来好像也不是特别有用的列。如果没有一些关于船票号码意义的专业领域知识和哪些名字的特点像大户或者有钱家庭，那么Ticket和Name列也不太可能会告诉我们太多的信息。

In [4]:
print(titanic.isnull().sum(axis=0))

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


## 性别处理
- 将0赋值给male，将1赋值给female

In [5]:
titanic.loc[titanic['Sex']=='male','Sex'] = 0
titanic.loc[titanic['Sex']=='female','Sex'] = 1
print(titanic.head(3))

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris   0  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...   1  38.0      1      0   
2                             Heikkinen, Miss. Laina   1  26.0      0      0   

             Ticket     Fare Cabin Embarked  
0         A/5 21171   7.2500   NaN        S  
1          PC 17599  71.2833   C85        C  
2  STON/O2. 3101282   7.9250   NaN        S  


## 登船Embarked列
- Embarked列的唯一值是S，C，Q和缺失值nan。每个字母是一个登船港口名字的缩写
- 将0赋值给S，将1赋值给C，将2赋值给Q

In [6]:
print(titanic['Embarked'].unique())
titanic['Embarked'] = titanic['Embarked'].fillna('S')
titanic.loc[titanic['Embarked'] == 'S', 'Embarked'] = 0
titanic.loc[titanic['Embarked'] == 'C', 'Embarked'] = 1
titanic.loc[titanic['Embarked'] == 'Q', 'Embarked'] = 2
print(titanic.head(3))

['S' 'C' 'Q' nan]
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris   0  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...   1  38.0      1      0   
2                             Heikkinen, Miss. Laina   1  26.0      0      0   

             Ticket     Fare Cabin Embarked  
0         A/5 21171   7.2500   NaN        0  
1          PC 17599  71.2833   C85        1  
2  STON/O2. 3101282   7.9250   NaN        0  


# 进行预测操作
- 使用scikit-learn库来做预测。用skelearn的一个助手来将数据分成交叉验证的子类，然后用每一个子类分别来训练算法做预测。最后，我们将得到一个预测列表，每一个列表项包含了相关子类的预测数据。

In [7]:
# Import the linear regression class
from sklearn.linear_model import LinearRegression
# Sklearn also has a helper that makes it easy to do cross validation
from sklearn.model_selection import KFold

# The columns we'll use to predict the target
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

# Initialize our algorithm class
alg = LinearRegression()
# Generate cross validation folds for the titanic dataset.  It return the row indices corresponding to train and test.
# We set random_state to ensure we get the same splits every time we run this.
kf = KFold(n_splits=3, random_state=1)

predictions = []
for train, test in kf.split(titanic['Age']):
    # The predictors we're using the train the algorithm.  Note how we only take the rows in the train folds.
    train_predictors = (titanic[predictors].iloc[train,:])
    # The target we're using to train the algorithm.
    train_target = titanic["Survived"].iloc[train]
    # Training the algorithm using the predictors and target.
    alg.fit(train_predictors, train_target)
    # We can now make predictions on the test fold
    test_predictions = alg.predict(titanic[predictors].iloc[test,:])
    predictions.append(test_predictions)

# 处理测试集

In [8]:
titanic_test = pd.read_csv("../input/test.csv")
titanic_test['Age'] = titanic_test['Age'].fillna(titanic_test['Age'].median())
titanic_test['Fare'] = titanic_test['Fare'].fillna(titanic_test['Fare'].median())
titanic_test.loc[titanic_test['Sex'] == 'male','Sex'] = 0
titanic_test.loc[titanic_test['Sex'] == 'female','Sex'] = 1
titanic_test['Embarked'] = titanic_test['Embarked'].fillna('S')
titanic_test.loc[titanic_test['Embarked'] == 'S', 'Embarked'] = 0
titanic_test.loc[titanic_test['Embarked'] == 'C', 'Embarked'] = 1
titanic_test.loc[titanic_test['Embarked'] == 'Q', 'Embarked'] = 2

NameError: name 'pandas' is not defined

# 生成一个提交文件
- 首先，我们必须先在训练数据上训练一个算法。然后我们在测试数据集上做一个预测。最后，我们生成一个包含预测和乘客id的csv文件。
- 一旦你完成了下面步骤的代码，你就能用submission.to_csv('submission.csv', index=False)输出一个csv结果。

In [ ]:
# Initialize the algorithm class
alg = LogisticRegression(random_state=1)

# Train the algorithm using all the training data
alg.fit(titanic[predictors], titanic["Survived"])

# Make predictions using the test set.
predictions = alg.predict(titanic_test[predictors])

In [ ]:
predictions = predictions.astype(int)
# Create a new dataframe with only the columns Kaggle wants from the dataset.
submission = pandas.DataFrame({
        "PassengerId": titanic_test["PassengerId"],
        "Survived": predictions
    })
submission.to_csv('Data/submission.csv', index=False)

# 完整代码

In [ ]:
#!/usr/bin/env python
# coding=utf-8
# Filename : first.py
# Created by iFantastic on 2017/8/14
# Description : http://www.cnblogs.com/cutd/p/5713520.html

import pandas as pd
# Import the linear regression class
from sklearn.linear_model import LinearRegression
# Sklearn also has a helper that makes it easy to do cross validation
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

# 读取训练数据
titanic = pd.read_csv('../input/train.csv')

# 处理Age中的缺失值
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].median())
# 量化Sex
titanic.loc[titanic['Sex'] == 'male', 'Sex'] = 0
titanic.loc[titanic['Sex'] == 'female', 'Sex'] = 1
# 量化Embarked
titanic['Embarked'] = titanic['Embarked'].fillna('S')
titanic.loc[titanic['Embarked'] == 'S', 'Embarked'] = 0
titanic.loc[titanic['Embarked'] == 'C', 'Embarked'] = 1
titanic.loc[titanic['Embarked'] == 'Q', 'Embarked'] = 2

# 进行预测操作
# The columns we'll use to predict the target
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

# Initialize our algorithm class
alg = LinearRegression()
# Generate cross validation folds for the titanic dataset.  It return the row indices corresponding to train and test.
# We set random_state to ensure we get the same splits every time we run this.
folder = KFold(n_splits=3, random_state=1)

predictions = []
for train, test in folder.split(titanic[predictions]):
    # The predictors we're using the train the algorithm.  Note how we only take the rows in the train folds.
    train_predictors = (titanic[predictors].iloc[train, :])
    # The target we're using to train the algorithm.
    train_target = titanic["Survived"].iloc[train]
    # Training the algorithm using the predictors and target.
    alg.fit(train_predictors, train_target)
    # We can now make predictions on the test fold
    test_predictions = alg.predict(titanic[predictors].iloc[test, :])
    predictions.append(test_predictions)

# 处理测试集
titanic_test = pd.read_csv("../input/test.csv")
titanic_test['Age'] = titanic_test['Age'].fillna(titanic_test['Age'].median())
titanic_test['Fare'] = titanic_test['Fare'].fillna(titanic_test['Fare'].median())
titanic_test.loc[titanic_test['Sex'] == 'male', 'Sex'] = 0
titanic_test.loc[titanic_test['Sex'] == 'female', 'Sex'] = 1
titanic_test['Embarked'] = titanic_test['Embarked'].fillna('S')
titanic_test.loc[titanic_test['Embarked'] == 'S', 'Embarked'] = 0
titanic_test.loc[titanic_test['Embarked'] == 'C', 'Embarked'] = 1
titanic_test.loc[titanic_test['Embarked'] == 'Q', 'Embarked'] = 2

# 用测试集进行预测
# Initialize the algorithm class
alg = LogisticRegression(random_state=1)

# Train the algorithm using all the training data
alg.fit(titanic[predictors], titanic["Survived"])

# Make predictions using the test set.
predictions = alg.predict(titanic_test[predictors])

# 保存结果
# Create a new dataframe with only the columns Kaggle wants from the dataset.
predictions = predictions.astype(int)
submission = pd.DataFrame({
        "PassengerId": titanic_test["PassengerId"],
        "Survived": predictions
    })
submission.to_csv('../input/submission.csv', index=False)